In [27]:
import pandas as pd

In [2]:
df = pd.read_excel('marketing.xlsx')

In [3]:
df.head()

,No,Year_Birth,Age_Customers,Education,Marital_Status,Income,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,NumWebPurchases,NumWebVisitsMonth,AcceptedCmp1,AcceptedCmp2,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5
0,1,1970,44,Graduation,Divorced,84.835,2014-06-16,0,189,104,379,111,4,1,0,0,0,0,0
1,2,1961,53,Graduation,Single,57.091,2014-06-15,0,464,5,64,7,7,5,0,1,0,0,0
2,3,1958,56,Graduation,Married,67.267,2014-05-13,0,134,11,59,15,3,2,0,0,0,0,0
3,4,1967,47,Graduation,In Relationship,32.474,2014-11-05,0,10,0,1,0,1,7,0,0,0,0,0
4,5,1989,25,Graduation,Single,21.474,2014-08-04,0,6,16,24,11,3,7,0,0,1,0,0


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2216 entries, 0 to 2215
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   No                 2216 non-null   int64         
 1   Year_Birth         2216 non-null   int64         
 2   Age_Customers      2216 non-null   int64         
 3   Education          2216 non-null   object        
 4   Marital_Status     2216 non-null   object        
 5   Income             2216 non-null   float64       
 6   Dt_Customer        2216 non-null   datetime64[ns]
 7   Recency            2216 non-null   int64         
 8   MntWines           2216 non-null   int64         
 9   MntFruits          2216 non-null   int64         
 10  MntMeatProducts    2216 non-null   int64         
 11  MntFishProducts    2216 non-null   int64         
 12  NumWebPurchases    2216 non-null   int64         
 13  NumWebVisitsMonth  2216 non-null   int64         
 14  Accepted

In [10]:
df_recency = df[['No','Recency']]
df_recency.head()

,No,Recency
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [12]:
frequency_df = df[['No', 'NumWebVisitsMonth']]
frequency_df.head()

,No,NumWebVisitsMonth
0,1,1
1,2,5
2,3,2
3,4,7
4,5,7


In [13]:
df['Monetary'] = df['MntWines']+df['MntFruits']+df['MntMeatProducts']+df['MntFishProducts']
monetary_df = df[['No', 'Monetary']]
monetary_df.head()

,No,Monetary
0,1,783
1,2,540
2,3,219
3,4,11
4,5,57


In [14]:
rf_df = df_recency.merge(frequency_df, on='No')
rfm_df = rf_df.merge(monetary_df, on='No')
rfm_df.head()

,No,Recency,NumWebVisitsMonth,Monetary
0,1,0,1,783
1,2,0,5,540
2,3,0,2,219
3,4,0,7,11
4,5,0,7,57


In [16]:
rfm_df['R_rank'] = rfm_df['Recency'].rank(ascending=False)
rfm_df['F_rank'] = rfm_df['NumWebVisitsMonth'].rank(ascending=True)
rfm_df['M_rank'] = rfm_df['Monetary'].rank(ascending=True)
 
# normalizing the rank of the customers
rfm_df['R_rank_norm'] = (rfm_df['R_rank']/rfm_df['R_rank'].max())*100
rfm_df['F_rank_norm'] = (rfm_df['F_rank']/rfm_df['F_rank'].max())*100
rfm_df['M_rank_norm'] = (rfm_df['F_rank']/rfm_df['M_rank'].max())*100
 
rfm_df.drop(columns=['R_rank', 'F_rank', 'M_rank'], inplace=True)
 
rfm_df.head()

,No,Recency,NumWebVisitsMonth,Monetary,R_rank_norm,F_rank_norm,M_rank_norm
0,1,0,1,783,100.0,3.860045,3.858303
1,2,0,5,540,100.0,41.580135,41.561372
2,3,0,2,219,100.0,11.783296,11.777978
3,4,0,7,11,100.0,71.738149,71.705776
4,5,0,7,57,100.0,71.738149,71.705776


In [17]:
rfm_df['RFM_Score'] = 0.15*rfm_df['R_rank_norm']+0.28 * \
    rfm_df['F_rank_norm']+0.57*rfm_df['M_rank_norm']
rfm_df['RFM_Score'] *= 0.05
rfm_df = rfm_df.round(2)
rfm_df[['No', 'RFM_Score']].head(7)

,No,RFM_Score
0,1,0.91
1,2,2.52
2,3,1.25
3,4,3.80
4,5,3.80
5,6,1.25
6,7,3.11


In [18]:
import numpy as np

In [19]:
rfm_df["Customer_segment"] = np.where(rfm_df['RFM_Score'] >
                                      4.5, "Top Customers",
                                      (np.where(
                                        rfm_df['RFM_Score'] > 4,
                                        "High value Customer",
                                        (np.where(
    rfm_df['RFM_Score'] > 3,
                             "Medium Value Customer",
                             np.where(rfm_df['RFM_Score'] > 1.6,
                            'Low Value Customers', 'Lost Customers'))))))
rfm_df[['No', 'RFM_Score', 'Customer_segment']].head(20)

,No,RFM_Score,Customer_segment
0,1,0.91,Lost Customers
1,2,2.52,Low Value Customers
2,3,1.25,Lost Customers
3,4,3.80,Medium Value Customer
4,5,3.80,Medium Value Customer
5,6,1.25,Lost Customers
6,7,3.11,Medium Value Customer
7,8,2.52,Low Value Customers
8,9,2.04,Low Value Customers
9,10,2.04,Low Value Customers


In [20]:
rfm_df.Customer_segment.unique()

array(['Lost Customers', 'Low Value Customers', 'Medium Value Customer',
       'High value Customer', 'Top Customers'], dtype=object)

In [21]:
df2 = df.merge(rfm_df, on='No')

In [22]:
df2

,No,Year_Birth,Age_Customers,Education,Marital_Status,Income,Dt_Customer,Recency_x,MntWines,MntFruits,...,AcceptedCmp5,Monetary_x,Recency_y,NumWebVisitsMonth_y,Monetary_y,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment
0,1,1970,44,Graduation,Divorced,84.835,2014-06-16,0,189,104,...,0,783,0,1,783,100.00,3.86,3.86,0.91,Lost Customers
1,2,1961,53,Graduation,Single,57.091,2014-06-15,0,464,5,...,0,540,0,5,540,100.00,41.58,41.56,2.52,Low Value Customers
2,3,1958,56,Graduation,Married,67.267,2014-05-13,0,134,11,...,0,219,0,2,219,100.00,11.78,11.78,1.25,Lost Customers
3,4,1967,47,Graduation,In Relationship,32.474,2014-11-05,0,10,0,...,0,11,0,7,11,100.00,71.74,71.71,3.80,Medium Value Customer
4,5,1989,25,Graduation,Single,21.474,2014-08-04,0,6,16,...,0,57,0,7,57,100.00,71.74,71.71,3.80,Medium Value Customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2211,2212,1976,38,PhD,Divorced,66.476,2013-07-03,99,372,18,...,0,563,99,4,563,0.41,30.38,30.37,1.29,Lost Customers
2212,2213,1977,37,2n Cycle,Married,31.056,2013-01-22,99,5,10,...,0,31,99,8,31,0.41,88.15,88.11,3.75,Medium Value Customer
2213,2214,1976,38,Graduation,Divorced,46.310,2012-03-12,99,185,2,...,0,290,99,8,290,0.41,88.15,88.11,3.75,Medium Value Customer
2214,2215,1978,36,Graduation,Married,65.819,2012-11-29,99,267,38,...,0,1155,99,3,1155,0.41,20.90,20.89,0.89,Lost Customers


In [24]:
df2['Age_Group'] = np.where((df2['Age_Customers'] >=
                                      18) & (df2['Age_Customers'] <= 24), "Youth",
                                      (np.where(
                                        (df2['Age_Customers'] > 24) & (df2['Age_Customers'] <= 44),
                                        "Adults",
                                        (np.where(
    (df2['Age_Customers'] > 44) & (df2['Age_Customers'] <= 64),
                             "Seniors",
                             np.where(df2['Age_Customers'] > 64,
                            'Elderly', 'else'))))))

In [25]:
df2.head(5)

,No,Year_Birth,Age_Customers,Education,Marital_Status,Income,Dt_Customer,Recency_x,MntWines,MntFruits,...,Monetary_x,Recency_y,NumWebVisitsMonth_y,Monetary_y,R_rank_norm,F_rank_norm,M_rank_norm,RFM_Score,Customer_segment,Age_Group
0,1,1970,44,Graduation,Divorced,84.835,2014-06-16,0,189,104,...,783,0,1,783,100.0,3.86,3.86,0.91,Lost Customers,Adults
1,2,1961,53,Graduation,Single,57.091,2014-06-15,0,464,5,...,540,0,5,540,100.0,41.58,41.56,2.52,Low Value Customers,Seniors
2,3,1958,56,Graduation,Married,67.267,2014-05-13,0,134,11,...,219,0,2,219,100.0,11.78,11.78,1.25,Lost Customers,Seniors
3,4,1967,47,Graduation,In Relationship,32.474,2014-11-05,0,10,0,...,11,0,7,11,100.0,71.74,71.71,3.80,Medium Value Customer,Seniors
4,5,1989,25,Graduation,Single,21.474,2014-08-04,0,6,16,...,57,0,7,57,100.0,71.74,71.71,3.80,Medium Value Customer,Adults


In [26]:
df2.to_excel('marketing_fix.xlsx')